In [1]:
import fenics as fe
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Mesh and Finite Element Discretization
N_POINTS_P_AXIS = 128
mesh = fe.UnitSquareMesh(N_POINTS_P_AXIS, N_POINTS_P_AXIS)
lagrange_polynomial_space_first_order = fe.FunctionSpace(mesh, "Lagrange", 1)

# Boundary Conditions
def boundary_boolean_function(x, on_boundary): 
    return on_boundary

homogeneous_dirichlet_boundary_condition = fe.DirichletBC(
    lagrange_polynomial_space_first_order,
    fe.Constant(0.0),
    boundary_boolean_function,
)

# The initial condition
parameter = 1
x0, y0 = 0.5, 0.5  # Center of the initial condition in the unit square
magnitude = 1
initial_condition = fe.Expression("magnitude*(1-exp(-1/(parameter*parameter*(pow(x[0] - x0, 2) + pow(x[1] - y0, 2)))))", 
                                  degree=2, magnitude=magnitude, parameter=parameter, x0=x0, y0=y0, domain=mesh)

# Discretize the initial condition
u_old = fe.interpolate(initial_condition, lagrange_polynomial_space_first_order)

# Parameters for the Gaussian function
A = 1.0          # Peak amplitude
x0, y0 = 0.5, 0.5  # Center of the Gaussian in the unit square
sigma = 0.1       # Standard deviation (controls the spread)

# Define the Gaussian diffusion coefficient
D = fe.Expression("A * exp(-((x[0] - x0)*(x[0] - x0) + (x[1] - y0)*(x[1] - y0)) / (2 * sigma))",
                  degree=2, A=A, x0=x0, y0=y0, sigma=sigma, domain=mesh)

# Time-stepping parameters
time_step_length = 0.001
n_time_steps = 200

#Create the finite element problem
u_trial = fe.TrialFunction(lagrange_polynomial_space_first_order)
v_test = fe.TestFunction(lagrange_polynomial_space_first_order)

weak_form_residuum = (
    (u_trial * v_test * fe.dx
    +
    time_step_length * (fe.dot(fe.grad(u_trial), fe.grad(D)) * v_test
                        +
                        D * fe.dot(fe.grad(u_trial), fe.grad(v_test))) * fe.dx)
    -
    u_old * v_test * fe.dx
    -
    time_step_length * fe.dot(fe.grad(D), fe.grad(u_trial)) * v_test * fe.dx
)

#We have a linear PDE that is separable into a lhs and rhs
weak_form_lhs = fe.lhs(weak_form_residuum)
weak_form_rhs = fe.rhs(weak_form_residuum)

# Solution function for each timestep
u_solution = fe.Function(lagrange_polynomial_space_first_order)

# Set up file for saving results
vtkfile = fe.File("Gaussian_Solution/solution.pvd")

# Lists to store time and solution at the point of interest
times = []
point_values = []

# Time-stepping loop
time_current = 0.0
for i in range(n_time_steps):

    # Solve the PDE for the current time step
    fe.solve(weak_form_lhs == weak_form_rhs, u_solution, homogeneous_dirichlet_boundary_condition)

    # Update the previous solution
    u_old.assign(u_solution)
    
    # Save the current solution to the .pvd file
    vtkfile << (u_solution, time_current)
    
    time_current += time_step_length

Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational problem.
Solving linear variational p